<a href="https://colab.research.google.com/github/Shahorukh/Student-Grading-and-Performance-Analysis-System/blob/main/BPP_3_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Data Initialization
Create a CSV file to store initial student data with the following fields:
Student ID, Name, Subjects (e.g., Math, English, Science), Marks, Grade.
Store the data in Python data structures like lists or dictionaries for processing.



In [4]:
import csv
data = list()
with open("/content/student_data.csv", mode ="r")as file:
  csvFile = csv.reader(file)
  for lines in csvFile:
    data.append(lines)
    print(lines)

['Student ID', 'Name', 'Subjects', 'Marks', 'Grade']
['1', 'Sharukh', 'Math, English, Science', 'Math: 95, English: 88, Science: 92', '']
['2', 'Minhaz', 'Math, English, Science', 'Math: 78, English: 85, Science: 80', '']
['3', 'Habib', 'Math, English, Science', 'Math: 65, English: 70, Science: 72', '']
['4', 'Foysal', 'Math,English,Science', 'Math:75,  English:80,  Science:85', '']


# New section